In [6]:
%matplotlib inline
import torch
import torchvision
from torch.utils import data
from torchvision import transforms

In [41]:
# 读取数据集
trans = transforms.ToTensor()
mnist_train_dataset = torchvision.datasets.FashionMNIST(root="./data",train=True,transform=trans,download=True)
mnist_test_dataset = torchvision.datasets.FashionMNIST(root="./data",train=False,transform=trans,download=True)

In [42]:
#送入数据迭代器
batch_size = 256
train_iter = data.DataLoader(mnist_train_dataset,batch_size,shuffle=True,num_workers=0)
test_iter = data.DataLoader(mnist_test_dataset,batch_size,shuffle=False,num_workers=0)

In [25]:
# 定义函数，该函数用于获取数据集迭代器
def load_dataset_iter(batch_size,resize=None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0,transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train_dataset = torchvision.datasets.FashionMNIST(root="./data",train=True,transform=trans,download=True)
    mnist_test_dataset = torchvision.datasets.FashionMNIST(root="./data",train=False,transform=trans,download=True)
    train_iter = data.DataLoader(mnist_train_dataset,batch_size,shuffle=True,num_workers=0)
    test_iter = data.DataLoader(mnist_test_dataset,batch_size,shuffle=False,num_workers=0)
    return (train_iter,test_iter)

In [38]:
train_iter,test_iter = load_dataset_iter(32,64)
for X,y in train_iter:
    print(y)
    break

tensor([9, 3, 6, 1, 2, 4, 2, 4, 8, 6, 7, 3, 9, 4, 8, 9, 2, 0, 7, 6, 4, 7, 0, 7,
        1, 1, 8, 9, 6, 7, 9, 3])


In [28]:
from IPython import display
batch_size = 256
train_iter,test_iter = load_dataset_iter(batch_size)
# 初始化参数
num_inputs = 784
num_outputs = 10
w = torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)
b = torch.zeros(num_outputs,requires_grad=True)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

In [37]:
# 定义softmax
def softmax(y):
    y_exp = torch.exp(y)
    y_exp_sum = y_exp.sum(dim=1,keepdim=True)
    return y_exp / y_exp_sum
# 验证
y = torch.Tensor([[1,2,3],[4,5,6]])
print(float(softmax(y).argmax(axis=1).sum()))

4.0


In [ ]:
# 定义模型
def net(X,w,b):
    y_hat = softmax(torch.matmul(X.reshape((-1,w.shape[0])),w) + b)
    return y_hat

In [34]:
# 定义损失函数
def cross_entropy(y_hat,y):
    loss_list = -torch.log(y_hat[range(len(y_hat)),y])
    return loss_list.mean()

tensor([[ 2,  4,  6],
        [ 5,  7,  9],
        [ 8, 10, 12]])

In [45]:
# 定义优化器
lr = 0.1
def updata(params,lr):
    with torch.no_grad(): # 确保在参数更新过程中不会生成新的计算图
        for param in params:
            param -= lr * param.grad
            param.grad.zero_()

In [43]:
def accuracy(y_hat,y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1: # 判断一下如果是多分类问题就执行argmax操作，如果不是则不需要额外处理
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [44]:
# 定义累加器
class Accumulator:
    """在n个变量上累加"""
    def __init__(self,n):
        self.data = [0.0] * n
    def add(self,*args):
        self.data = [a + float(b) for a,b in zip(self.data,args)]
    def reset(self):
        self.data = [0.0] * len(self.data)
    def __getitem__(self,idx):
        return self.data[idx]

In [ ]:
# 定义一个训练函数来训练一个迭代周期
def train_epoch(net,train_dataiter,loss,updater):
    if isinstance(net,torch.nn.Module):
        net.eval()
    metric = Accumulator(3)
    for X,y in train_dataiter:
        y_hat = net(X,w,b)
        l = loss(y_hat,y)
        l.backward()
        updater([w,b],lr)
        metric.add(float(l),accuracy(y_hat,y),y.numel())
        
    return metric[0],metric[1] / metric[2]

In [ ]:
# 计算精度
def evaluate_accuracy(net,data_iter):
    """利用指定模型在指定数据迭代器上计算精度"""
    if isinstance(net,torch.nn.Module):
        net.eval()
    metric = Accumulator(2)  # 正确预测数、预测总数
    with torch.no_grad():
        for X,y in data_iter:
            metric.add(accuracy(net(X,w,b),y),y.numel())
    return metric[0] / metric[1]
        

In [10]:
import numpy as np
features = np.random.normal(size=(200, 1))
features

array([[-1.00318445e-01],
       [ 1.52192212e+00],
       [-8.31270688e-01],
       [ 1.84528664e-02],
       [ 3.68733920e-01],
       [ 1.06162542e+00],
       [-2.76523625e-01],
       [ 5.90222360e-01],
       [ 1.89555368e+00],
       [ 1.37945086e+00],
       [ 2.94169012e-01],
       [-3.93348001e-01],
       [-2.44536248e-02],
       [ 1.72092127e+00],
       [ 6.43324816e-01],
       [ 1.49307128e+00],
       [ 1.03650132e+00],
       [ 6.06256100e-01],
       [ 7.08438509e-01],
       [-9.87666230e-01],
       [-1.74383601e-01],
       [-6.26924877e-01],
       [-6.54038377e-01],
       [-6.94656026e-01],
       [ 3.51871134e-01],
       [ 1.05575357e+00],
       [-5.60400136e-01],
       [-1.08621011e+00],
       [ 7.03240080e-04],
       [-1.12440849e+00],
       [ 1.41337039e+00],
       [-9.63413033e-01],
       [-3.51025264e-01],
       [-3.84433930e-02],
       [-2.35675556e-01],
       [-8.19032168e-01],
       [ 4.52046841e-03],
       [-9.93429031e-01],
       [ 4.5

In [18]:
np.random.shuffle(features)
poly_features = np.power(features, np.arange(20).reshape(1, -1))
poly_features

array([[ 1.00000000e+00, -2.96176974e-01,  8.77207998e-02, ...,
        -1.03841739e-09,  3.07555320e-10, -9.10908038e-11],
       [ 1.00000000e+00,  9.11988361e-01,  8.31722771e-01, ...,
         2.08842090e-01,  1.90461555e-01,  1.73698722e-01],
       [ 1.00000000e+00,  4.42324323e-01,  1.95650807e-01, ...,
         9.49716473e-07,  4.20082696e-07,  1.85812794e-07],
       ...,
       [ 1.00000000e+00,  9.79640709e-01,  9.59695919e-01, ...,
         7.04913795e-01,  6.90562250e-01,  6.76502892e-01],
       [ 1.00000000e+00, -5.72566638e-01,  3.27832555e-01, ...,
        -7.63910822e-05,  4.37389851e-05, -2.50434836e-05],
       [ 1.00000000e+00,  1.89555368e+00,  3.59312375e+00, ...,
         5.26639565e+04,  9.98273565e+04,  1.89228113e+05]])

In [30]:
import numpy as np
import math
print(math.gamma(5))

24.0
